In [2]:
import numpy as np
import util

from linear_model import LinearModel

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
class LogisticRegression(LinearModel):
    """Logistic regression with Newton's Method as the solver.

    Example usage:
        > clf = LogisticRegression()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def fit(self, x, y, method='gradient'):
        """Fit model using either Gradient Ascent or Newton's Method.

        Args:
            x: Training inputs (m, n).
            y: Training labels (m,).
            method: Optimization method ('gradient' or 'newton').
        """
        # *** START CODE HERE ***
        m, n = x.shape
        self.theta = np.zeros(n) if self.theta is None else self.theta

        if method == 'gradient':
            for _ in range(self.max_iter):
                h = sigmoid(x @ self.theta)
                gradient = x.T @ (y - h) / m
                self.theta += self.step_size * gradient

        elif method == 'newton':
            for i in range(self.max_iter):
                h = sigmoid(x @ self.theta)
                gradient = x.T @ (y - h) / m
                S = np.diag(h * (1 - h))  # (m, m)
                H = (x.T @ S @ x) / m     # (n, n)

                try:
                    delta = np.linalg.solve(H, gradient)
                except np.linalg.LinAlgError:
                    print("Warning: Hessian not invertible.")
                    break

                self.theta += delta

                # convergence check: stop if delta is small
                if np.linalg.norm(delta, ord=1) < self.eps:
                    if self.verbose:
                        print(f"Converged at iteration {i}")
                    break
        else:
            raise ValueError("Method must be 'gradient' or 'newton'.")
        # *** END CODE HERE ***

    def predict(self, x):
        """Make a prediction given new inputs x.

        Args:
            x: Inputs of shape (m, n).

        Returns:
            Outputs of shape (m,).
        """
        # *** START CODE HERE ***
        return (sigmoid(x @ self.theta) > 0.5).astype(int)
        # *** END CODE HERE ***


In [5]:
train_path = '/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds1_train.csv'
eval_path = '/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds1_valid.csv'
# def main(train_path, eval_path, pred_path):
"""Problem 1(b): Logistic regression with Newton's Method.

Args:
    train_path: Path to CSV file containing dataset for training.
    eval_path: Path to CSV file containing dataset for evaluation.
    pred_path: Path to save predictions.
"""
x_train, y_train = util.load_dataset(train_path, add_intercept=True)

x_eval, y_eval = util.load_dataset(eval_path, add_intercept=True)

In [6]:

# *** START CODE HERE ***
clf = LogisticRegression()
clf.fit(x_train, y_train, method='gradient')
prediction = clf.predict(x_eval)
print(prediction)
print(y_eval)
print((prediction-y_eval).sum())
# *** END CODE HERE ***

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1
 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
-36.0


/var/folders/f9/kqml6rjn5g5688g1y7fn01ch0000gn/T/ipykernel_30413/1253679003.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


## GDA

In [7]:
import numpy as np
import util

from linear_model import LinearModel


def main(train_path, eval_path, pred_path):
    """Problem 1(e): Gaussian discriminant analysis (GDA)

    Args:
        train_path: Path to CSV file containing dataset for training.
        eval_path: Path to CSV file containing dataset for evaluation.
        pred_path: Path to save predictions.
    """
    # Load dataset
    x_train, y_train = util.load_dataset(train_path, add_intercept=False)

    # *** START CODE HERE ***
    # *** END CODE HERE ***


class GDA(LinearModel):
    """Gaussian Discriminant Analysis.

    Example usage:
        > clf = GDA()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """

    def fit(self, x, y):
        """Fit a GDA model to training set given by x and y.

        Args:
            x: Training example inputs. Shape (m, n).
            y: Training example labels. Shape (m,).

        Returns:
            theta: GDA model parameters.
        """
        # *** START CODE HERE ***
        m, n = x.shape
        phi = np.mean(y)

        mu0 = x[y == 0].mean(axis=0)
        mu1 = x[y == 1].mean(axis=0)
        self.mu = [mu0, mu1]

        # Shared covariance
        diffs = x - np.where(y[:, np.newaxis] == 1, mu1, mu0)
        sigma = (diffs.T @ diffs) / m
        self.sigma = sigma

        # Inverse
        sigma_inv = np.linalg.inv(sigma)

        # Compute theta_1 and theta_0
        theta_1 = sigma_inv @ (mu1 - mu0)
        theta_0 = (
            0.5 * (mu0 @ sigma_inv @ mu0 - mu1 @ sigma_inv @ mu1)
            - np.log((1 - phi) / phi)
        )

        # Final theta with intercept
        self.theta = np.zeros(n + 1)
        self.theta[0] = theta_0
        self.theta[1:] = theta_1
        # *** END CODE HERE ***

    def predict(self, x, method='theta'):
        """Make a prediction given new inputs x.

        Args:
            x: Inputs of shape (m, n).
            method: theta | naive

        Returns:
            Outputs of shape (m,).
        """
        # *** START CODE HERE ***
        def px_y(x, sigma, mu):
            """
            x: shape (m, n)
            mu: shape (n,)
            sigma: shape (n, n)
            returns: shape (m,)
            """
            m, n = x.shape
            inv_sigma = np.linalg.inv(sigma)
            det_sigma = np.linalg.det(sigma)
            norm_const = 1.0 / ((2 * np.pi) ** (n / 2) * np.sqrt(det_sigma))

            # Center x
            centered = x - mu  # shape (m, n)
            
            # Compute Mahalanobis distance for each row: (x - mu)^T @ Sigma^-1 @ (x - mu)
            # Using einsum for efficient row-wise dot product
            exponent = -0.5 * np.einsum("ij,jk,ik->i", centered, inv_sigma, centered)

            return norm_const * np.exp(exponent)

        if method == "theta":
            x = util.add_intercept(x)
            return 1 / (1 + np.exp(-x @ self.theta))
        elif method == "naive":
            px_y1 = px_y(x, self.sigma, self.mu[1])
            px_y0 = px_y(x, self.sigma, self.mu[0])
            p_y1 = 0.5  # (y_train == 1).sum() / m
            p_y0 = 0.5  # (y_train == 0).sum() / m
            prediction = (px_y1 * p_y1) / (px_y1*p_y1 + px_y0*p_y0)
            return prediction
        # *** END CODE HERE


In [8]:
train_path = '/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds1_train.csv'
x_train, y_train = util.load_dataset(train_path, add_intercept=False)

In [9]:
x_train[0:10]

array([[  2.91180854,  60.35961272],
       [  3.77474554, 344.1492843 ],
       [  2.61548828, 178.22208681],
       [  2.01369376,  15.25947155],
       [  2.75762504,  66.19417399],
       [  0.97392246,  41.67766519],
       [  3.06727469, 143.27558992],
       [  2.76309408,  35.96990594],
       [  2.7757715 ,  29.56907921],
       [  2.10982995,  76.63672138]])

In [10]:
clf = GDA()
clf.fit(x_train, y_train)
clf.predict(x_train[440:442], method="naive")

array([0.96595743, 0.85103215])

<img src="./phi_gda_estimation.png">
<br/>
<img src="./mean_gda_estimation.png">
<br/>
<img src="./sigma_gda_estiomation.png">


In [11]:

m = len(x_train)
phi = y_train.sum() / m

mu0 = np.sum(x_train[np.where(y_train == 0)], axis=0) / ((y_train == 0).sum())

mu1 = np.sum(x_train[np.where(y_train == 1)], axis=0) / ((y_train == 1).sum())

mu = np.array([mu0, mu1])

diffs = x_train - mu[y_train.astype(int)]
sigma = (diffs.T @ diffs) / m

<img src="./gda_prediction.png">

In [12]:
def px_y(x, sigma, mu):
    return np.exp((-1/2) * (x - mu).T @ np.linalg.inv(sigma) @ (x - mu)) / ((np.pi ** (2 / 2)) * np.sqrt(np.linalg.det(sigma))) 

In [13]:
xx = x_train[751]

In [14]:
y_train[451]

np.float64(1.0)

In [15]:
px_y1 = px_y(xx, sigma, mu[1])
px_y0 = px_y(xx, sigma, mu[0])
p_y1 = (y_train == 1).sum() / m
p_y0 = (y_train == 0).sum() / m
prediction = (px_y1 * p_y1) / (px_y1*p_y1 + px_y0*p_y0)
prediction

np.float64(0.9846356805435632)

## Positive-Unlabeled Learning

In [16]:
from p01b_logreg import LogisticRegression
train_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds3_train.csv"
test_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds3_test.csv"

In [17]:
x_train_t, y_train_t = util.load_dataset(train_path, label_col='t')
x_train_y, y_train_y = util.load_dataset(train_path, label_col='y')

x_test_t, y_test_t = util.load_dataset(test_path, label_col='t')
x_test_y, y_test_y = util.load_dataset(test_path, label_col='y')

# Part (c): Train and test on true labels
# Make sure to save outputs to pred_path_c

# Use newton method with t labels
clf = LogisticRegression()
clf.fit(x=x_train_t, y=y_train_t, method='newton')
clf.predict(x_test_t)

Converged at iteration 8


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [20]:
train_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds4_train.csv"
val_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds4_valid.csv"
x_train, y_train = util.load_dataset(train_path, add_intercept=True)

In [42]:
x_train[0].shape, theta.shape

((5,), (5,))

In [45]:
res = x_train @ theta
np.exp(res)

array([1., 1., 1., ..., 1., 1., 1.], shape=(2500,))

In [76]:
x = x_train[i]
y = y_train[i]
logits = h(x @ theta)
print("logits", logits)

logits 1.0


In [86]:

np.array([y]) @ x.T

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 1)

In [87]:
x.T.shape, np.array([y]).shape, logits.shape

((5,), (1,), ())

In [73]:
gradient = x.T @ (y - logits)


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [71]:
m, n = x_train.shape
theta = np.zeros(n)
def h(x):
    return np.exp(x)

epoch = 20
for i in range(len(x_train)):
    print("theta", theta)
    x = x_train[i]
    y = y_train[i].item()
    logits = h(x @ theta)
    print("logits", logits)
    gradient = x.T @ (y - logits)
    print("gradient", gradient)
    theta += 0.000000001 * gradient

theta [0. 0. 0. 0. 0.]
logits 1.0


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [57]:
x_train[i], theta

(array([1.        , 0.        , 1.        , 0.22499177, 0.88080895]),
 array([-23780.43032319,  -1529.38993991, -22251.04038328, -19894.26250972,
        -20447.73674632]))

In [55]:
i = 4
h((x_train[i] @ theta)), y_train[i]

(np.float64(0.0), np.float64(3705439.0))

In [165]:
import numpy as np
import util

from linear_model import LinearModel


def main(lr, train_path, eval_path, pred_path):
    """Problem 3(d): Poisson regression with gradient ascent.

    Args:
        lr: Learning rate for gradient ascent.
        train_path: Path to CSV file containing dataset for training.
        eval_path: Path to CSV file containing dataset for evaluation.
        pred_path: Path to save predictions.
    """
    # Load training set
    x_train, y_train = util.load_dataset(train_path, add_intercept=True)
    # The line below is the original one from Stanford. It does not include the intercept, but this should be added.
    # x_train, y_train = util.load_dataset(train_path, add_intercept=False)

    # *** START CODE HERE ***
    # Fit a Poisson Regression model
    clf = PoissonRegression(step_size=0.1e-7, max_iter=200)
    clf.fit(x_train, y_train)
    # Run on the validation set, and use np.savetxt to save outputs to pred_path
    x_val, y_val = util.load_dataset(val_path, add_intercept=True)
    preds = clf.predict(x_val)
    np.savetxt(pred_path, preds)
    # *** END CODE HERE ***


class PoissonRegression(LinearModel):
    """Poisson Regression.

    Example usage:
        > clf = PoissonRegression(step_size=lr)
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """

    def fit(self, x, y):
        """Run gradient ascent to maximize likelihood for Poisson regression.

        Args:
            x: Training example inputs. Shape (m, n).
            y: Training example labels. Shape (m,).
        """
        # *** START CODE HERE ***
        m, n = x.shape
        self.theta = np.zeros(n)
        def h(x):
            return np.exp(x)

        epoch = self.max_iter
        for _ in range(epoch):
            for i in range(len(x)):
                logits = h(x[i] @ self.theta)
                gradient = x[i].reshape((x[i].shape[0], 1)) @ np.array([y[i] - logits])
                self.theta += self.step_size * gradient        

        # *** END CODE HERE ***

    def predict(self, x):
        """Make a prediction given inputs x.

        Args:
            x: Inputs of shape (m, n).

        Returns:
            Floating-point prediction for each input, shape (m,).
        """
        # *** START CODE HERE ***
        return h(x @ self.theta)
        # *** END CODE HERE ***


In [159]:
train_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds4_train.csv"
val_path = "/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds4_valid.csv"
x_train, y_train = util.load_dataset(train_path, add_intercept=True)
x_val, y_val = util.load_dataset(val_path, add_intercept=True)

In [163]:
0.1e-7

1e-08

In [166]:
clf = PoissonRegression(step_size=0.1e-7, max_iter=200)
clf.fit(x_train, y_train)
clf.predict(x_val)

array([12003128.81595367,  2830162.83790876, 20527360.15875288,
        1606376.58911739,   478596.13620045,  5703974.36112889,
         688888.38769774,   482388.0033151 ,  1907967.57361079,
         113363.10624469,  4191656.6579308 ,   496518.39024542,
        4190044.98514974,  1847738.35948409,   146387.87794461,
        1168147.59610179,  1589893.80579298,  3150536.55234231,
         407892.12436554,   123136.27307857,  2118015.69987693,
         118232.56520093,  1183632.38984027,  4787284.55023211,
         578867.05136255,  9574790.17495857,   409630.2898261 ,
        1099321.39230494,  1744530.59255483,  1470552.07095193,
         495153.31735969,  3468862.12607031,  5675893.27632901,
       24961881.89916041,   465147.97110652, 14628494.51991394,
         777519.35942133,  1288812.11509628,   454300.94405375,
       10173635.83240312,  2684760.71138495,  7235160.74197726,
         460142.8572708 ,   203729.76500251,  3452957.46968923,
       10846931.97677287,  6562785.18271

In [5]:
x = x_train
y = y_train

In [30]:
x @ theta

array([203654.62911845, 194349.88006475, 190127.50206897, ...,
       204259.99388508, 242291.16667128, 188213.7258473 ], shape=(2500,))

In [33]:
logits = h(x @ theta)
logits
gradient = x.T @ (y - logits)
print(theta)
theta += 0.000000001 * gradient
print(theta)


[0. 0. 0. 0. 0.]
[8.81357768 3.52076832 5.29280936 5.79967485 6.99194722]


In [98]:
h((x[i] @ theta))
np.array([1])
# np.view(x[i], (1, 5))
xT = x[i].reshape((x[i].shape[0], 1))
xT

array([[1.        ],
       [1.        ],
       [0.        ],
       [0.99204749],
       [0.32574102]])

In [118]:
import math


In [131]:
log_likelihood = y[i] * (theta @ x[i]) - np.exp(theta @ x[i]) - np.log(np.array([math.factorial(y[i])]))

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [121]:
loss = y * (x@theta) - h(x@theta) # - np.log(math.factorial(y))
loss

array([3.37861291e+07, 7.98960297e+06, 5.72229559e+06, ...,
       1.30367137e+07, 1.61198107e+08, 6.82876138e+06], shape=(2500,))

In [148]:
m, n = x.shape
theta = np.zeros(n)
def h(x):
    return np.exp(x)

epoch = 20
for _ in range(epoch):
    for i in range(len(x)):
        logits = h(x[i] @ theta)
        gradient = x[i].reshape((x[i].shape[0], 1)) @ np.array([y[i] - logits])
        theta += 0.00000001 * gradient
    rlogits = h(x @ theta)
    loss = -(y * np.log(rlogits) - rlogits).mean()
    print(f"theta: {theta}")    
    print(f"loss: {loss}")    
    

theta: [7.24900432 3.87791461 3.37108971 2.04987811 4.56785931]
loss: -52196486.40377476
theta: [7.35470024 3.9276774  3.42702284 2.00379563 4.41781981]
loss: -52198874.8192148
theta: [7.36539386 3.93275208 3.43264179 2.00032669 4.40176443]
loss: -52198901.20656444
theta: [7.36647453 3.93326498 3.43320955 1.99997737 4.40014024]
loss: -52198901.51555419
theta: [7.36658354 3.93331671 3.43326683 1.9999421  4.39997642]
loss: -52198901.522623315
theta: [7.36659453 3.93332193 3.4332726  1.99993855 4.3999599 ]
loss: -52198901.52309103
theta: [7.36659564 3.93332245 3.43327318 1.99993819 4.39995823]
loss: -52198901.52313571
theta: [7.36659575 3.93332251 3.43327324 1.99993815 4.39995806]
loss: -52198901.523140185
theta: [7.36659576 3.93332251 3.43327325 1.99993815 4.39995805]
loss: -52198901.52314064
theta: [7.36659576 3.93332251 3.43327325 1.99993815 4.39995804]
loss: -52198901.523140684
theta: [7.36659576 3.93332251 3.43327325 1.99993815 4.39995804]
loss: -52198901.523140684
theta: [7.36659576

In [153]:
i = 6
logits = h(x @ theta)
(y - logits)
# theta

array([-661.84271855,  509.99086539,  463.81740282, ...,  240.31516983,
        527.14997129,  613.99668473], shape=(2500,))

# LWR

<img src="lwr.png">

In [10]:
import util
import numpy as np
x_train, y_train = util.load_dataset("/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds5_train.csv")
x_test, y_test = util.load_dataset("/Users/musazenbilci/Desktop/mosesopposite/cs229-2018-autumn/problem-sets/PS1/data/ds5_test.csv")

In [103]:
for t in range(len(x_test)):
    n = len(x_train)
    W = np.zeros((n, n))
    T = 0.5
    # t = 8
    xt = x_test[t]
    yt = y_test[t]
    y_train = y_train.reshape(-1, 1)
    for i in range(n):
        W[i][i] = np.exp((-(xt - x_train[i][0])**2) / (2 * T**2)).item()

    theta = np.linalg.inv(x_train.T @ W @ x_train) @ (x_train.T @ W @ y_train)
    print(np.abs((xt * theta)[0].item() - yt.item()))

0.2668126980371306
0.9939254742242551
1.0251723736942564
0.1840991956104546
1.1046836612534503
0.006630152307551651
0.15157919663094913
1.3847125119859336
0.9047927668850024
0.9391076559640481
0.7902746272191268
0.0031967116256148994
0.7420577185005341
0.4707860505340508
0.25901121461666476
0.8830621752618671
0.0673944647825232
0.5455072976014387
0.9373208174025089
0.4964294761816516
0.0951337481537371
0.3180855648341377
0.20843366496814228
0.6932920369293933
0.8871902718187191
0.758638994754532
0.7949801422898414
0.34529071761406926
0.7963226377115491
0.9450685083728148
0.24176695705517565
0.6065674215143295
0.2728120369454359
0.157762502787968
0.1547442001570197
0.29709270518323494
0.11302456402638385
0.9938068693028574
0.2694576651430219
0.14438167843645772
0.5177646519725173
0.20821712783136712
0.2663409805347371
0.4802418864149762
0.4031442411037825
0.26503653497532265
0.36904473098724033
0.12585723549243732
0.7905826325068395
0.6822001322577501
0.1253533228359066
0.38063303597723

In [92]:
y_train.shape, W.shape, (x_train.T @ W @ y_train).shape

((300,), (300, 300), (1,))

In [80]:
theta = np.linalg.inv(x_train.T @ W @ x_train) @ (x_train.T @ W @ y_train) #.reshape(y_train.shape[0], 1)
theta

array([-0.06463599])

In [81]:
x_test[0] @ theta, y_test[0]

(np.float64(0.108092528472029), np.float64(0.3749052265091596))

In [69]:
x_test.shape, theta.shape

((200, 1), (300, 1))

In [26]:
x_train[0:4]

array([[-4.02975001],
       [-2.52457661],
       [ 2.7946174 ],
       [ 1.69590357]])

In [46]:
x_train[0:2].shape, W[0:2].shape, (x_train[0:2] * W[0:2]).shape

((2, 1), (2,), (2, 2))

In [57]:
W = W.reshape(W.shape[0], 1)
x_train.shape, W.shape, (x_train*W).shape

((300, 1), (300, 1), (300, 1))

In [55]:
x1 = np.array([[1], [2], [3]])
y1 = np.array([1, 2, 3])
y1 = y1.reshape(y1.shape[0], 1)
x1.shape, y1.shape,  x1 * y1

((3, 1),
 (3, 1),
 array([[1],
        [4],
        [9]]))

In [ ]:
x_train * W * x_train

In [119]:
from linear_model import LinearModel

class LocallyWeightedLinearRegression(LinearModel):
    """Locally Weighted Regression (LWR).

    Example usage:
        > clf = LocallyWeightedLinearRegression(tau)
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """

    def __init__(self, tau):
        super(LocallyWeightedLinearRegression, self).__init__()
        self.tau = tau
        self.x = None
        self.y = None

    def fit(self, x, y):
        """Fit LWR by saving the training set.

        """
        # *** START CODE HERE ***
        self.x = x
        self.y = y
        # *** END CODE HERE ***

    def predict(self, x):
        """Make predictions given inputs x.

        Args:
            x: Inputs of shape (m, n).

        Returns:
            Outputs of shape (m,).
        """
        # *** START CODE HERE ***
        predictions = []
        for xt in x:
            n = len(self.x)
            W = np.zeros((n, n))
            for i in range(n):
                W[i][i] = np.exp((-(xt - self.x[i])**2) / (2 * self.tau**2)).item()

            theta = np.linalg.inv(self.x.T @ W @ self.x) @ (self.x.T @ W @ self.y)
            predictions.append(xt * theta)
        return np.array(predictions).flatten()
        # *** END CODE HERE ***


In [122]:
clf = LocallyWeightedLinearRegression(0.5)
clf.fit(x_train, y_train)
results = clf.predict(x_test)
results

array([ 0.10809253, -0.02320168,  0.06857   ,  0.24415736, -0.04129074,
        0.35886127,  0.18822755, -0.30857419,  0.08016958, -0.01196518,
        0.31726153,  0.00320507,  0.02137835,  0.07459675, -0.27168437,
       -0.14456169,  0.46229952,  0.09158327,  0.06559322, -0.34874019,
        0.07342643, -0.13554041, -0.0194822 , -0.21523376, -0.09733929,
       -0.0984498 , -0.02116349, -0.0362843 , -0.08295422, -0.29257002,
        0.48106296,  0.46228092, -0.01925451,  0.07597857,  0.09040299,
        0.57791596,  0.07947224,  0.06538456, -0.39128041,  0.4202518 ,
       -0.02531267, -0.02606352, -0.00825445, -0.07533382, -0.05699588,
        0.57797871,  0.42373586,  0.44763394,  0.23185159, -0.07420662,
        0.57697982,  0.41978752,  0.01478812,  0.02597826,  0.33216602,
        0.29875778,  0.44493812, -0.09198549, -0.02875572,  0.54845131,
        0.5387758 , -0.03453282, -0.09521599,  0.07044772, -0.09801627,
        0.02751528, -0.30339894, -0.05011201,  0.0865589 ,  0.24